# IMPORTAMOS LA LIBRERIAS NECESARIAS

In [1]:
from glob import glob
import pandas as pd

### Cargamos la data

In [2]:
df_metadata = pd.read_parquet('../../data/raw/g-sitios.parquet')

In [3]:
# Funsion para hacer un explode en aquellas columnas que contienen listas de datos.
def explode_columna(df: pd.DataFrame, columna_a_explodar: str) -> pd.DataFrame:
    return df.explode(columna_a_explodar)
# Funsion para filtrar la categoria que vayamos a usar de acuerdo al tipo de estudio de eleccion.
def filtrar_por_categoria(df: pd.DataFrame, columna_a_filtrar: str, keyword: str) -> pd.DataFrame:
    df = df.dropna(subset=[columna_a_filtrar])
    return df[df[columna_a_filtrar].str.contains(keyword, case=False)]

In [4]:
df_metadata.head(2)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.35710,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.21529,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [5]:
df_nuevo = explode_columna(df_metadata,'category')

In [6]:
# Definir el patrón regex para extraer la dirección, ciudad, estado y código postal
patron = r'([^,]+),\s*([^,]+),\s*([^,]+),\s*([^,]+)$'

# Aplicar el regex a cada entrada de la columna "address" y almacenar los resultados en columnas separadas
resultados = df_nuevo['address'].str.extract(patron)

# Asignar los resultados a nuevas columnas en el DataFrame
df_nuevo['Local'] = resultados[0]
df_nuevo['Direccion'] = resultados[1]
df_nuevo['Ciudad'] = resultados[2]
df_nuevo['Codigo_Postal'] = resultados[3]

# Insertar las nuevas columnas justo después de la columna "address"
df_nuevo.insert(df_nuevo.columns.get_loc('address') + 1, 'Local', df_nuevo.pop('Local'))
df_nuevo.insert(df_nuevo.columns.get_loc('address') + 2, 'Direccion', df_nuevo.pop('Direccion'))
df_nuevo.insert(df_nuevo.columns.get_loc('address') + 3, 'Ciudad', df_nuevo.pop('Ciudad'))
df_nuevo.insert(df_nuevo.columns.get_loc('address') + 4, 'Codigo_Postal', df_nuevo.pop('Codigo_Postal'))

In [7]:
df_nuevo.head(2)

,name,address,Local,Direccion,Ciudad,Codigo_Postal,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",Porter Pharmacy,129 N Second St,Cochran,GA 31014,0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.35710,Pharmacy,4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",City Textile,3001 E Pico Blvd,Los Angeles,CA 90023,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.21529,Textile exporter,4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [8]:
df_nuevo.drop(columns=["address"], inplace=True)

In [9]:
df_nuevo.drop(columns=["Local"], inplace=True)

In [10]:
# Dividir la columna "Codigo_Postal" en dos nuevas columnas: "Estado" y "Codigo_Postal"
df_nuevo[['Estado', 'Codigo_Postal']] = df_nuevo['Codigo_Postal'].str.split(' ', n=1, expand=True)

In [11]:
df_nuevo.head(5)

,name,Direccion,Ciudad,Codigo_Postal,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,Estado
0,Porter Pharmacy,129 N Second St,Cochran,31014,0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,Pharmacy,4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...,GA
1,City Textile,3001 E Pico Blvd,Los Angeles,90023,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,Textile exporter,4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...,CA
2,San Soo Dang,761 S Vermont Ave,Los Angeles,90005,0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,Korean restaurant,4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...,CA
3,Nova Fabrics,2200 E 11th St,Los Angeles,90021,0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,Fabric store,3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...,CA
4,Nobel Textile Co,719 E 9th St,Los Angeles,90021,0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,Fabric store,4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...,CA


In [12]:
# Crear una lista de los estados con los prefijos deseados
estados_prefijos = ['PA', 'NJ', 'DE']

# Filtrar el DataFrame para incluir solo las filas que tengan estados con los prefijos deseados
df_filtrado = df_nuevo[df_nuevo['Estado'].isin(estados_prefijos)]

In [13]:
df_filtrado.head(3)

,name,Direccion,Ciudad,Codigo_Postal,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,Estado
57,Primo Pizza & Subs,148 W St Joseph St,Easton,18042,0x89c46d5e4554eae1:0xa2f8b211524ca29a,None,40.682177,-75.209820,Pizza delivery,4.5,25,$$,"[[Thursday, 11AM–9PM], [Friday, 11AM–10PM], [S...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 9PM,"[0x89c46c56cac9c661:0xdc9d0dd3f2b203ae, 0x89c4...",https://www.google.com/maps/place//data=!4m2!3...,PA
62,S.J. O’Hara Inc.,143 White Oak Ct,Tannersville,18372,0x89c48c790b767ec7:0x1943c20a42196a68,None,41.055487,-75.284536,HVAC contractor,4.5,8,None,"[[Thursday, 7AM–7PM], [Friday, 7AM–7PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 7PM,"[0x89c48c7bd96b49f9:0xdf562ab873031e1, 0x89c45...",https://www.google.com/maps/place//data=!4m2!3...,PA
62,S.J. O’Hara Inc.,143 White Oak Ct,Tannersville,18372,0x89c48c790b767ec7:0x1943c20a42196a68,None,41.055487,-75.284536,Air conditioning contractor,4.5,8,None,"[[Thursday, 7AM–7PM], [Friday, 7AM–7PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 7PM,"[0x89c48c7bd96b49f9:0xdf562ab873031e1, 0x89c45...",https://www.google.com/maps/place//data=!4m2!3...,PA


In [14]:
# Contar los valores repetidos en la columna "Estado"
conteo_estados = df_filtrado['Estado'].value_counts()

# Mostrar el conteo de estados repetidos
print(conteo_estados)

Estado
PA    225362
NJ    157246
DE     17920
Name: count, dtype: int64


In [15]:
# Seleccionar solo algunas columnas en un nuevo DataFrame
nuevo_df = df_filtrado[['name', 'gmap_id','Direccion','Ciudad','Estado', 'Codigo_Postal','description','latitude','longitude','category','avg_rating','num_of_reviews','price','hours','MISC',	'state','relative_results','url']]


In [16]:
nuevo_df.head(3)

,name,gmap_id,Direccion,Ciudad,Estado,Codigo_Postal,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
57,Primo Pizza & Subs,0x89c46d5e4554eae1:0xa2f8b211524ca29a,148 W St Joseph St,Easton,PA,18042,None,40.682177,-75.209820,Pizza delivery,4.5,25,$$,"[[Thursday, 11AM–9PM], [Friday, 11AM–10PM], [S...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 9PM,"[0x89c46c56cac9c661:0xdc9d0dd3f2b203ae, 0x89c4...",https://www.google.com/maps/place//data=!4m2!3...
62,S.J. O’Hara Inc.,0x89c48c790b767ec7:0x1943c20a42196a68,143 White Oak Ct,Tannersville,PA,18372,None,41.055487,-75.284536,HVAC contractor,4.5,8,None,"[[Thursday, 7AM–7PM], [Friday, 7AM–7PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 7PM,"[0x89c48c7bd96b49f9:0xdf562ab873031e1, 0x89c45...",https://www.google.com/maps/place//data=!4m2!3...
62,S.J. O’Hara Inc.,0x89c48c790b767ec7:0x1943c20a42196a68,143 White Oak Ct,Tannersville,PA,18372,None,41.055487,-75.284536,Air conditioning contractor,4.5,8,None,"[[Thursday, 7AM–7PM], [Friday, 7AM–7PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 7PM,"[0x89c48c7bd96b49f9:0xdf562ab873031e1, 0x89c45...",https://www.google.com/maps/place//data=!4m2!3...


In [17]:
nuevo_df.reset_index(drop=True, inplace=True)

In [20]:
nuevo_df.to_parquet('../../data/metadata_google.parquet')

In [23]:
df_ciudades = pd.read_parquet('../../data/y_business_CLEAN.parquet')

In [24]:
df_ciudades.head(2)

,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,hours
business_id,,,,,,,,,,,
kfNv-JZpuN6TVNSO6hHdkw,Hibachi Express,6625 E 82nd St,Indianapolis,PA,46250,39.904320,-86.053080,4.0,20,"[Steakhouses, Asian Fusion, Restaurants]","{'Friday': '11:0-21:0', 'Monday': '11:0-21:0',..."
aPNXGTDkf-4bjhyMBQxqpQ,Craft Hall,901 N Delaware Ave,Philadelphia,PA,19123,39.962582,-75.135657,3.5,65,"[Eatertainment, Arts & Entertainment, Brewpubs...","{'Friday': '16:0-19:0', 'Monday': '0:0-0:0', '..."


In [28]:
# Convertir todas las ciudades a minúsculas y luego obtener una lista de ciudades únicas
ciudades_unicas = df_ciudades['city'].str.lower().unique().tolist()

In [29]:
print(ciudades_unicas)

['indianapolis', 'philadelphia', 'glenolden', 'wesley chapel', 'santa barbara', 'new orleans', 'reno', 'saint louis', 'tucson', 'nashville', 'madison', 'st. louis', 'king of prussia', 'clearwater', 'tampa', 'st albert', 'saint petersburg', 'franklin', 'brownsburg', 'maple shade', 'lansdale', 'oldsmar', 'langhorne', 'isla vista', 'newtown', 'woodstown', 'plant city', 'boise', 'spring hill', 'conshohocken', 'lebanon', 'sherwood park', 'southampton', 'largo', 'edmonton', 'mount juliet', 'turnersville', 'doylestown', 'bensalem', 'wilmington', 'wayne', 'moorestown', 'st louis', 'washington crossing', 'zionsville', 'harahan', 'old hickory', 'narberth', 'west chester', 'pitman', 'seminole', 'malvern', 'cherry hill', 'port richey', 'springfield', 'smyrna', 'riverview', 'st petersburg', 'woolwich twp', 'trooper', 'meridian', 'north wales', 'dunedin', 'harleysville', 'bala cynwyd', 'metairie', 'bryn mawr', 'willow grove', 'chadds ford', 'whiteland', 'belleville', 'brandon', 'st pete beach', 'ash

In [30]:
# Convertir todas las ciudades de la columna "Ciudad" a minúsculas
nuevo_df['Ciudad'] = nuevo_df['Ciudad'].str.lower()

In [32]:
# Filtrar el DataFrame original para incluir solo las filas con ciudades que están en la lista de ciudades únicas
df_filtrado2 = nuevo_df[nuevo_df['Ciudad'].isin(ciudades_unicas)]

In [33]:
df_filtrado2.head(5)

,name,gmap_id,Direccion,Ciudad,Estado,Codigo_Postal,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
8,Vivi Bubble Tea,0x89c6c89efcaed69d:0xded973f6033e7dba,701 S 5th St,philadelphia,PA,19147,"Bubble tea shop offering hot & cold versions, ...",39.940293,-75.150923,Restaurant,4.0,8,None,None,"{'Accessibility': None, 'Activities': None, 'A...",None,"[0x89c6c62a40f4b733:0x73bfeec1898f272b, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...
9,Kater Street Bicycle,0x89c6c63c8cd87141:0x54d0d283872eecbb,609 S 16th St,philadelphia,PA,19146,None,39.943673,-75.168762,Bicycle Shop,4.9,15,None,"[[Thursday, 11AM–7PM], [Friday, 2–6PM], [Satur...",None,Permanently closed,"[0x89c6c623342d6197:0x70233738cd92d9b, 0x89c6c...",https://www.google.com/maps/place//data=!4m2!3...
10,Kater Street Bicycle,0x89c6c63c8cd87141:0x54d0d283872eecbb,609 S 16th St,philadelphia,PA,19146,None,39.943673,-75.168762,Bicycle repair shop,4.9,15,None,"[[Thursday, 11AM–7PM], [Friday, 2–6PM], [Satur...",None,Permanently closed,"[0x89c6c623342d6197:0x70233738cd92d9b, 0x89c6c...",https://www.google.com/maps/place//data=!4m2!3...
11,Cadence Cycling and Cafe - Center City,0x89c6c715d1821fe3:0x9cfa8308c0ce2289,201 S 25th St,philadelphia,PA,19103,None,39.950078,-75.180729,Bicycle Shop,4.8,38,None,"[[Thursday, 7AM–7PM], [Friday, 7AM–7PM], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 7PM,"[0x89c6c7d1a79d4103:0x2476b0fe30eb0871, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...
12,Cadence Cycling and Cafe - Center City,0x89c6c715d1821fe3:0x9cfa8308c0ce2289,201 S 25th St,philadelphia,PA,19103,None,39.950078,-75.180729,Bicycle repair shop,4.8,38,None,"[[Thursday, 7AM–7PM], [Friday, 7AM–7PM], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 7PM,"[0x89c6c7d1a79d4103:0x2476b0fe30eb0871, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...


In [34]:
# Contar el número de ciudades únicas en el nuevo DataFrame
num_ciudades_unicas = df_filtrado2['Ciudad'].nunique()

# Mostrar el resultado
print("Número de ciudades únicas en el nuevo DataFrame:", num_ciudades_unicas)

Número de ciudades únicas en el nuevo DataFrame: 308


In [36]:
# Contar cuántos elementos hay en la lista de ciudades únicas
num_ciudades_unicas_lista = len(ciudades_unicas)

# Mostrar el resultado
print("Número de ciudades únicas en la lista:", num_ciudades_unicas_lista)

Número de ciudades únicas en la lista: 558


In [37]:
# Eliminar duplicados de la columna 'gmap_id'
df_sin_duplicados = df_filtrado2.drop_duplicates(subset=['gmap_id'])

# Resetear el índice del DataFrame resultante
df_sin_duplicados.reset_index(drop=True, inplace=True)


In [38]:
df_sin_duplicados.head(5)

,name,gmap_id,Direccion,Ciudad,Estado,Codigo_Postal,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Vivi Bubble Tea,0x89c6c89efcaed69d:0xded973f6033e7dba,701 S 5th St,philadelphia,PA,19147,"Bubble tea shop offering hot & cold versions, ...",39.940293,-75.150923,Restaurant,4.0,8,None,None,"{'Accessibility': None, 'Activities': None, 'A...",None,"[0x89c6c62a40f4b733:0x73bfeec1898f272b, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...
1,Kater Street Bicycle,0x89c6c63c8cd87141:0x54d0d283872eecbb,609 S 16th St,philadelphia,PA,19146,None,39.943673,-75.168762,Bicycle Shop,4.9,15,None,"[[Thursday, 11AM–7PM], [Friday, 2–6PM], [Satur...",None,Permanently closed,"[0x89c6c623342d6197:0x70233738cd92d9b, 0x89c6c...",https://www.google.com/maps/place//data=!4m2!3...
2,Cadence Cycling and Cafe - Center City,0x89c6c715d1821fe3:0x9cfa8308c0ce2289,201 S 25th St,philadelphia,PA,19103,None,39.950078,-75.180729,Bicycle Shop,4.8,38,None,"[[Thursday, 7AM–7PM], [Friday, 7AM–7PM], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 7PM,"[0x89c6c7d1a79d4103:0x2476b0fe30eb0871, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...
3,Indian Valley Closing Services Inc,0x89c6a024f46d6bc1:0x39a17e288d3ce9b5,504 Harleysville Pike,souderton,PA,18964,None,40.315358,-75.331928,Title company,5.0,6,None,"[[Thursday, 8:30AM–5PM], [Friday, 8:30AM–5PM],...",None,Open ⋅ Closes 5PM,"[0x89c6961c4f3fcc59:0x5f2b676cc9009204, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...
4,Glenside Bible Church,0x89c6b0839da68acb:0xdefc7bf5c5a05cdc,447 N Keswick Ave,glenside,PA,19038,None,40.110587,-75.149630,Church,4.7,14,None,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x89c6b09a5cf8ad0d:0x66d070c7641c2fe4, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...


In [39]:
df_sin_duplicados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56917 entries, 0 to 56916
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              56917 non-null  object 
 1   gmap_id           56917 non-null  object 
 2   Direccion         56917 non-null  object 
 3   Ciudad            56917 non-null  object 
 4   Estado            56917 non-null  object 
 5   Codigo_Postal     56917 non-null  object 
 6   description       5494 non-null   object 
 7   latitude          56917 non-null  float64
 8   longitude         56917 non-null  float64
 9   category          56632 non-null  object 
 10  avg_rating        56917 non-null  float64
 11  num_of_reviews    56917 non-null  int64  
 12  price             6213 non-null   object 
 13  hours             43600 non-null  object 
 14  MISC              44505 non-null  object 
 15  state             44333 non-null  object 
 16  relative_results  52968 non-null  object

In [40]:
# Contar los valores nulos en la columna 'Category'
valores_nulos_category = df_filtrado2['category'].isnull().sum()

# Mostrar el resultado
print("Número de valores nulos en la columna 'Category':", valores_nulos_category)

Número de valores nulos en la columna 'Category': 287


In [50]:
# Filtrar el DataFrame para mostrar solo las filas donde 'Category' es nulo
valores_nulos_df = df_sin_duplicados[df_sin_duplicados['category'].isnull()]

# Mostrar el DataFrame resultante
print(valores_nulos_df['category'])


250      None
501      None
667      None
687      None
811      None
         ... 
55362    None
55459    None
56312    None
56377    None
56573    None
Name: category, Length: 285, dtype: object


In [47]:
# Eliminar filas con valores nulos en la columna 'Category' y resetear el índice
df = df_sin_duplicados.dropna(subset=['category']).reset_index(drop=True)


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56632 entries, 0 to 56631
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              56632 non-null  object 
 1   gmap_id           56632 non-null  object 
 2   Direccion         56632 non-null  object 
 3   Ciudad            56632 non-null  object 
 4   Estado            56632 non-null  object 
 5   Codigo_Postal     56632 non-null  object 
 6   description       5484 non-null   object 
 7   latitude          56632 non-null  float64
 8   longitude         56632 non-null  float64
 9   category          56632 non-null  object 
 10  avg_rating        56632 non-null  float64
 11  num_of_reviews    56632 non-null  int64  
 12  price             6198 non-null   object 
 13  hours             43552 non-null  object 
 14  MISC              44505 non-null  object 
 15  state             44281 non-null  object 
 16  relative_results  52799 non-null  object

In [53]:
df.to_parquet("../../data/metadata_google.parquet")